In [1]:
from pangeo_forge_cmr import files_from_cmr
# rename to file_pattern_from_cmr
# to be more consistent <> pangeo-forge 


# Get the GPM IMERG Late Precipitation Daily data
shortname = 'GPM_3IMERGDL'

files = files_from_cmr( # Provide a list of files by querying CMR
        shortname,
        version='06',
        nitems_per_file=1,
        concat_dim='time',  # Describe how the dataset is chunked
    )


In [2]:
files

<FilePattern {'time': 8471}>

In [3]:
pattern_pruned = files.prune()
pattern_pruned
# only prunes in time, if pattern is small in time dimension, but large in another dimension

<FilePattern {'time': 2}>

In [4]:
import apache_beam as beam
from pangeo_forge_recipes.transforms import OpenURLWithFSSpec, OpenWithXarray, StoreToZarr

In [8]:
from tempfile import TemporaryDirectory
td = TemporaryDirectory()


In [17]:
import aiohttp
(username, account,password) = netrc().authenticators("urs.earthdata.nasa.gov")
client_kwargs = {
            "auth": aiohttp.BasicAuth(
                username, password
            ),
            "trust_env": True,
        }

In [20]:
transforms = (
    beam.Create(pattern_pruned.items())
    | OpenURLWithFSSpec(
        open_kwargs = {"client_kwargs": client_kwargs},
    )
    | OpenWithXarray(file_type=pattern_pruned.file_type)
    | StoreToZarr(
        target_root = td.name, # this would be the cloud bucket
        # this will be deleted when session ends or when kernal is restarted
        store_name = 'gpm.zarr', # wherever it goes this is what the name of the file will be
        combine_dims = pattern_pruned.combine_dim_keys,
    )
)
transforms

<_ChainedPTransform(PTransform) label=[Create|OpenURLWithFSSpec|OpenWithXarray|StoreToZarr] at 0x7f958196db50>

In [21]:
with beam.Pipeline() as p:
    p | transforms

In [26]:
import xarray as xr

full_path = os.path.join(td.name, "gpm.zarr")
ds = xr.open_dataset(full_path, engine="zarr")
ds

<xarray.Dataset>
Dimensions:                    (time: 2, lon: 3600, lat: 1800, nv: 2)
Coordinates:
  * lat                        (lat) float32 -89.95 -89.85 ... 89.85 89.95
  * lon                        (lon) float32 -179.9 -179.8 ... 179.9 179.9
  * time                       (time) object 2000-06-01 00:00:00 2000-06-02 0...
Dimensions without coordinates: nv
Data variables:
    HQprecipitation            (time, lon, lat) float32 ...
    HQprecipitation_cnt        (time, lon, lat) int8 ...
    HQprecipitation_cnt_cond   (time, lon, lat) int8 ...
    precipitationCal           (time, lon, lat) float32 ...
    precipitationCal_cnt       (time, lon, lat) int8 ...
    precipitationCal_cnt_cond  (time, lon, lat) int8 ...
    randomError                (time, lon, lat) float32 ...
    randomError_cnt            (time, lon, lat) int8 ...
    time_bnds                  (time, nv) object ...
Attributes:
    BeginTime:  00:00:00.000Z
    DOI:        10.5067/GPM/IMERGDL/DAY/06
    EndTime:    23:59:59.999Z
    title:      GPM IMERG Late Precipitation L3 1 day 0.1 degree x 0.1 degree...